In [1]:
import pandas as pd
import os
from pysam import VariantFile
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import seaborn as sns
from functools import reduce
from cyvcf2 import VCF
import pyupset as pyu
from upsetplot import generate_counts
from upsetplot import plot
import upsetplot

plt.style.use('aa_paper')
%matplotlib inline

def translate_gene_name(gene_name):
    """
    HDF5 throws all sort of errors when you have weird punctuation in the gene name, so
    this translates it to a less offensive form.
    """
    repls = ('-', 'dash'), ('.', 'period')
    trans_gene_name = reduce(lambda a, kv: a.replace(*kv), repls, str(gene_name))
    return trans_gene_name

# Reformat VCFs in parallel

`/pollard/home/kathleen/projects/AlleleAnalyzer/manuscript_analyses/wtc_analysis/src/hg19_analysis/reformat_variants_wtc_hg19_1.sh` (1-4) to parallelize 313762 regions defined in the 1000 Genomes hg19 region splitting.

Using a script called `get_gens_df.py` in `AlleleAnalyzer/generate_gens_dfs/get_gens_df.py`, we reformat the WTC VCF in order to more easily annotate variants for whether they are near or in PAM sites. This is necessary because in ordinary VCF files, variants can have multiple alleles listed on one line, and these need to be split up for annotation based on each individual allele. 

In [2]:
hg19_regions = pd.read_csv('../1000genomes_analysis/dat/1kgp_hg19_regions.bed', sep='\t', header=None,
                          names=['chrom','start','stop','region_id'])

In [3]:
hg19_regions['gens_fname'] = '/pollard/data/projects/AlleleAnalyzer_data/wtc_data/hg19/wtc_formatted_variants/' + hg19_regions['region_id'] + '.h5'

hg19_regions['gens_complete'] = hg19_regions['gens_fname'].map(os.path.isfile)

In [6]:
hg19_regions.query('~gens_complete')[~hg19_regions['chrom'].str.contains('_')]

/pollard/home/kathleen/tools/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


,chrom,start,stop,region_id,gens_fname,gens_complete


# Annotate variants in parallel

`/pollard/home/kathleen/projects/AlleleAnalyzer/manuscript_analyses/wtc_analysis/src/hg19_analysis/annotate_chrom_wtc_hg19.sh` requires that you have aggregates the variants by chromosome.

In [2]:
annots_store = pd.HDFStore('/pollard/data/projects/AlleleAnalyzer_data/AlleleAnalyzer_supporting_data/wtc_analysis/wtc_hg19_annots_all.h5')

In [6]:
annots_store.close()

In [10]:
annots = pd.read_hdf('/pollard/data/projects/AlleleAnalyzer_data/wtc_data/hg19/wtc_annotated_variants_by_chrom2/chr1.h5')

In [19]:
annots = pd.read_hdf('/pollard/data/projects/AlleleAnalyzer_data/wtc_data/hg19/wtc_annotated_variants_by_chrom/chr1_annotated.h5')

In [12]:
annots = pd.read_hdf('/pollard/data/projects/AlleleAnalyzer_data/AlleleAnalyzer_supporting_data/wtc_analysis/wtc_hg19_annots_all.h5', 'chrchr1',
                    where='pos >= 55505149 and pos <= 55530526')

In [8]:
reformat = pd.read_hdf('/pollard/data/projects/AlleleAnalyzer_data/wtc_data/hg19/wtc_formatted_variants_by_chrom2/chr1.h5')

In [3]:
# annots.query('(pos < 55530526) and (pos > 55505149)')

# ExcisionFinder analysis + single guide targetability analyses

Bash scripts in directory

In [2]:
genes = pd.read_csv('/pollard/home/kathleen/projects/AlleleAnalyzer/manuscript_analyses/1000genomes_analysis/get_gene_list/gene_list_hg19.tsv',
                   sep='\t')

Identify targetable genes per person by each method:

Dual-sgRNA approach:

`python /pollard/home/kathleen/projects/AlleleAnalyzer/manuscript_analyses/wtc_analysis/src/hg19_analysis/targ_genes_per_person.py /pollard/home/kathleen/projects/AlleleAnalyzer/manuscript_analyses/1000genomes_analysis/get_gene_list/gene_list_hg19.tsv /pollard/home/kathleen/projects/AlleleAnalyzer/manuscript_analyses/wtc_analysis/src/hg19_analysis/samples.txt /pollard/data/projects/AlleleAnalyzer_data/wtc_data/hg19/wtc_excisionfinder_results2/ /pollard/data/projects/AlleleAnalyzer_data/wtc_data/hg19/wtc_targ_genes_per_person/`

Single-sgRNA approach:

`python /pollard/home/kathleen/projects/AlleleAnalyzer/manuscript_analyses/wtc_analysis/src/hg19_analysis/targ_genes_per_person_single.py /pollard/home/kathleen/projects/AlleleAnalyzer/manuscript_analyses/1000genomes_analysis/get_gene_list/gene_list_hg19.tsv /pollard/home/kathleen/projects/AlleleAnalyzer/manuscript_analyses/wtc_analysis/src/hg19_analysis/samples.txt /pollard/data/projects/AlleleAnalyzer_data/wtc_data/hg19/wtc_single_targ2/ /pollard/data/projects/AlleleAnalyzer_data/wtc_data/hg19/wtc_targ_genes_per_person/single_targ`

Dual-sgRNA approach + 5kb:

`python /pollard/home/kathleen/projects/AlleleAnalyzer/manuscript_analyses/wtc_analysis/src/hg19_analysis/targ_genes_per_person.py /pollard/home/kathleen/projects/AlleleAnalyzer/manuscript_analyses/1000genomes_analysis/get_gene_list/gene_list_hg19.tsv /pollard/home/kathleen/projects/AlleleAnalyzer/manuscript_analyses/wtc_analysis/src/hg19_analysis/samples.txt /pollard/data/projects/AlleleAnalyzer_data/wtc_data/hg19/wtc_excisionfinder_results2_5kb/ /pollard/data/projects/AlleleAnalyzer_data/wtc_data/hg19/wtc_targ_genes_per_person/ef_5kb_dual`

Single-sgRNA approach:

`python /pollard/home/kathleen/projects/AlleleAnalyzer/manuscript_analyses/wtc_analysis/src/hg19_analysis/targ_genes_per_person_single.py /pollard/home/kathleen/projects/AlleleAnalyzer/manuscript_analyses/1000genomes_analysis/get_gene_list/gene_list_hg19.tsv /pollard/home/kathleen/projects/AlleleAnalyzer/manuscript_analyses/wtc_analysis/src/hg19_analysis/samples.txt /pollard/data/projects/AlleleAnalyzer_data/wtc_data/hg19/wtc_single_targ_5kb2/
/pollard/data/projects/AlleleAnalyzer_data/wtc_data/hg19/wtc_targ_genes_per_person/ef_5kb_single`

# Determine # putatively targetable genes in WTC

In [3]:
genes = pd.read_csv('/pollard/home/kathleen/projects/AlleleAnalyzer/manuscript_analyses/1000genomes_analysis/get_gene_list/gene_list_hg19.tsv',
                   sep='\t')
autosomal_genes = genes.query('(chrom != "chrX") and (chrom != "chrY")')
protein_coding_autosomal_genes = set(genes[genes['name'].str.startswith('NM')]['official_gene_symbol'].tolist())

In [4]:
targ_genes_per_person = np.load('/pollard/data/projects/AlleleAnalyzer_data/wtc_data/hg19/wtc_targ_genes_per_person/genes_per_person.npy').item()
targ_genes_per_person_single = np.load('/pollard/data/projects/AlleleAnalyzer_data/wtc_data/hg19/wtc_targ_genes_per_person/single_targgenes_per_person.npy').item()
targ_genes_per_person_5kb = np.load('/pollard/data/projects/AlleleAnalyzer_data/wtc_data/hg19/wtc_targ_genes_per_person/ef_5kb_dualgenes_per_person.npy').item()
targ_genes_per_person_single_5kb = np.load('/pollard/data/projects/AlleleAnalyzer_data/wtc_data/hg19/wtc_targ_genes_per_person/ef_5kb_singlegenes_per_person.npy').item()

In [5]:
ppl = []
num_targ_genes = []
cas = []

for key in targ_genes_per_person:
    ppl.append(key)
    num_targ_genes.append(len(protein_coding_autosomal_genes.intersection(set(targ_genes_per_person[key]))))
    
targ_genes_per_person_df = pd.DataFrame({'ppl':ppl, 'num_targ_genes':num_targ_genes})

targ_genes_per_person_df['perc_targ_genes'] = targ_genes_per_person_df['num_targ_genes'].divide(len(protein_coding_autosomal_genes)) * 100.0

targ_genes_per_person_df['perc_targ_genes'].mean()

67.10769900903844

In [6]:
ppl = []
num_targ_genes = []
cas = []

for key in targ_genes_per_person_5kb:
    ppl.append(key)
    num_targ_genes.append(len(protein_coding_autosomal_genes.intersection(set(targ_genes_per_person_5kb[key]))))
    
targ_genes_per_person_df = pd.DataFrame({'ppl':ppl, 'num_targ_genes':num_targ_genes})

targ_genes_per_person_df['perc_targ_genes'] = targ_genes_per_person_df['num_targ_genes'].divide(len(protein_coding_autosomal_genes)) * 100.0

targ_genes_per_person_df['perc_targ_genes'].mean()

78.72699553522814

In [7]:
genes.head()

,official_gene_symbol,name,chrom,txStart,txEnd,cdsStart,cdsEnd,exonCount,exonStarts,exonEnds,size
0,A1BG,NM_130786.3,chr19,58858171,58864865,58858387,58864803,8,"58858171,58858718,58861735,58862756,58863648,5...","58858395,58859006,58862017,58863053,58863921,5...",6694
1,A1BG-AS1,NR_015380.2,chr19,58863335,58866549,58866549,58866549,4,"58863335,58864744,58865079,58865734,","58864410,58864840,58865223,58866549,",3214
2,A1CF,NM_001198820.1,chr10,52559168,52645435,52566488,52610547,14,"52559168,52569653,52570799,52573616,52575765,5...","52566640,52569802,52570936,52573798,52576039,5...",86267
3,A2M,NM_001347424.1,chr12,9220303,9268825,9220418,9265102,36,"9220303,9220778,9221335,9222340,9223083,922495...","9220435,9220820,9221438,9222409,9223174,922508...",48522
4,A2M-AS1,NR_026971.1,chr12,9217772,9220651,9220651,9220651,3,"9217772,9218421,9218751,","9217824,9218656,9220651,",2879


In [8]:
ppl = []
num_targ_genes = []
cas = []

for key in targ_genes_per_person_single:
    ppl.append(key)
    num_targ_genes.append(len(protein_coding_autosomal_genes.intersection(set(targ_genes_per_person_single[key]))))
    
targ_genes_per_person_df = pd.DataFrame({'ppl':ppl, 'num_targ_genes':num_targ_genes})

targ_genes_per_person_df['perc_targ_genes'] = targ_genes_per_person_df['num_targ_genes'].divide(len(protein_coding_autosomal_genes)) * 100.0

targ_genes_per_person_df['perc_targ_genes'].mean()

32.718065991506045

In [9]:
ppl = []
num_targ_genes = []
cas = []

for key in targ_genes_per_person_single_5kb:
    ppl.append(key)
    num_targ_genes.append(len(protein_coding_autosomal_genes.intersection(set(targ_genes_per_person_single_5kb[key]))))
    
targ_genes_per_person_df = pd.DataFrame({'ppl':ppl, 'num_targ_genes':num_targ_genes})

targ_genes_per_person_df['perc_targ_genes'] = targ_genes_per_person_df['num_targ_genes'].divide(len(protein_coding_autosomal_genes)) * 100.0

targ_genes_per_person_df['perc_targ_genes'].mean()

32.718065991506045

# Compare single to dual targeting in WTC

In [10]:
protein_coding_autosomal_genes_df = genes[genes['name'].str.startswith('NM')].query('(chrom != "chrX") and (chrom != "chrY")').copy()

In [11]:
vcf = VCF('/pollard/home/kathleen/conklin_wt_seq_data/wtc_wgs_data/phased_yin/wtc_PASS_hg19.phased.vcf.gz')

## Get number of hets per gene in WTC

In [12]:
n_hets_per_gene = {}

for ix, row in protein_coding_autosomal_genes_df.iterrows():
    gene_chrom = row['chrom']#.replace('chr','')
    gene_start = row['txStart']
    gene_stop = row['txEnd']
#     gene_vars = vcf(f'{gene_chrom}:{gene_start}-{gene_stop}')
    gene_vars = vcf(gene_chrom + ':' + str(gene_start) + '-' + str(gene_stop))
    gene_name = row['official_gene_symbol']
    n_hets = 0
    for var in gene_vars:
        hap1, hap2, whatever = var.genotypes[0]
        if hap1 != hap2:
            n_hets += 1
        else:
            continue
    n_hets_per_gene[gene_name] = n_hets

In [13]:
n_hets_per_gene_df = pd.DataFrame.from_dict(n_hets_per_gene, orient='index')
n_hets_per_gene_df['gene'] = n_hets_per_gene_df.index
n_hets_per_gene_df.columns = ['n_hets','gene']

In [14]:
less_two_hets = n_hets_per_gene_df.query('n_hets < 2')['gene'].tolist()
single_targ = targ_genes_per_person_single['WTC']
dual_targ = targ_genes_per_person['WTC']

In [15]:
upset_plot_dict = {'< 2 Hets': protein_coding_autosomal_genes_df[protein_coding_autosomal_genes_df['official_gene_symbol'].isin(less_two_hets)].reset_index(drop=True),
                  'Single': protein_coding_autosomal_genes_df[protein_coding_autosomal_genes_df['official_gene_symbol'].isin(single_targ)].reset_index(drop=True),
                  'Dual': protein_coding_autosomal_genes_df[protein_coding_autosomal_genes_df['official_gene_symbol'].isin(dual_targ)].reset_index(drop=True)}

# upset_plot_dict['< 2 Heterozygous Variants'] = n

In [37]:
protein_coding_autosomal_genes_df.head()

,official_gene_symbol,name,chrom,txStart,txEnd,cdsStart,cdsEnd,exonCount,exonStarts,exonEnds,size
0,A1BG,NM_130786.3,chr19,58858171,58864865,58858387,58864803,8,"58858171,58858718,58861735,58862756,58863648,5...","58858395,58859006,58862017,58863053,58863921,5...",6694
2,A1CF,NM_001198820.1,chr10,52559168,52645435,52566488,52610547,14,"52559168,52569653,52570799,52573616,52575765,5...","52566640,52569802,52570936,52573798,52576039,5...",86267
3,A2M,NM_001347424.1,chr12,9220303,9268825,9220418,9265102,36,"9220303,9220778,9221335,9222340,9223083,922495...","9220435,9220820,9221438,9222409,9223174,922508...",48522
5,A2ML1,NM_144670.5,chr12,8975067,9029379,8975247,9027607,36,"8975067,8975777,8976315,8982322,8987257,898810...","8975309,8975961,8976478,8982375,8987278,898826...",54312
7,A3GALT2,NM_001080438.1,chr1,33772366,33786699,33772366,33786699,5,"33772366,33777652,33778101,33778407,33786676,","33773054,33777790,33778191,33778491,33786699,",14333


In [45]:
genes_out = []
genes = []

for ix, row in protein_coding_autosomal_genes_df.iterrows():
    out = []
    for cat in ['< 2 Hets','Single','Dual']:
        if row['official_gene_symbol'] in upset_plot_dict[cat]['official_gene_symbol'].tolist():
            out.append(cat)
        else:
            continue
        genes_out.append(out)
        genes.append(row['official_gene_symbol'])

In [46]:
example

cat0   cat1   cat2 
False  False  False      56
              True      283
       True   False    1279
              True     5882
True   False  False      24
              True       90
       True   False     429
              True     1957
Name: value, dtype: int64

In [48]:
plotdat = upsetplot.from_memberships(genes_out)

In [73]:
fig = plt.Figure()
out = plot(plotdat, fig)

/pollard/home/kathleen/miniconda3/lib/python3.7/site-packages/upsetplot/plotting.py:36: FutureWarning: From version 0.4, passing a Series as data with non-unqiue groups will raise an error unless subset_size="sum" or "count".
  FutureWarning)
/pollard/home/kathleen/miniconda3/lib/python3.7/site-packages/upsetplot/plotting.py:387: UserWarning: tight_layout : falling back to Agg renderer
  r = get_renderer(fig)


In [74]:
fig.savefig('single_vs_dual_wtc_hg19_new.pdf', dpi=300)